In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import sys, logging
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import re
import os
import turnstile
import gtfs
import heuristics

root = logging.getLogger()
root.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

from gcs_utils import gcs_util
gcs = gcs_util(bucket_path='mta_crowding_data')


In [2]:
## Loading pre-requisite files
stop_order_merged = pd.read_csv('data/stops.csv')
crosswalk = pd.read_csv('data/Master_crosswalk.csv')
station_to_station = pd.read_csv('data/station_to_station.csv')


In [3]:
## defining date range for estimation
LAST_TURNSTILE_DATE = datetime(year=2020,month=10,day=18)
DAYS_RANGE = 2

dates = [LAST_TURNSTILE_DATE - timedelta(days=i) for i in range(1,DAYS_RANGE +1)]
keep_dates = [re.sub('-','',str(d.date())) for d in dates]


In [4]:
## proceesing real-time GTFS data
df, dates = gtfs.get_schedule(LAST_TURNSTILE_DATE,DAYS_RANGE)
clean_df = gtfs.process_schedule(df, min(dates), max(dates), stop_order_merged)

In [12]:
clean_static_schedule.uid.value_counts()

939     61
944     61
1077    61
1240    61
938     61
        ..
9384     2
9548     2
9128     2
9804     2
9999     2
Name: uid, Length: 14257, dtype: int64

In [5]:
## proceesing static GTFS data
static_schedule = pd.read_csv('data/google_transit/stop_times.txt')
trips = pd.read_csv('data/google_transit/trips.txt')
stops = pd.read_csv('data/google_transit/stops.txt')
static_schedule = static_schedule.merge(trips,on='trip_id').merge(stops,on='stop_id')

last_year_start = min(dates) - timedelta(weeks=52)
last_year_end = max(dates) - timedelta(weeks=52)

clean_static_schedule = gtfs.process_static_schedule(static_schedule,last_year_start, last_year_end)

In [6]:
## Processing Turnstile data

## Current
turnstile_data_raw = turnstile._process_raw_data(turnstile.download_turnstile_data(start_date=min(dates), end_date=max(dates)), group_by=['STATION','LINENAME','UNIT'])
turnstile_data_raw_imputed = turnstile.pre_interpolation_fix(turnstile_data_raw)
turnstile_data_raw_imputed = turnstile_data_raw_imputed.set_index('datetime')
turnstile_data = turnstile._interpolate(turnstile_data_raw_imputed, group_by=['STATION','LINENAME','UNIT'],  frequency='1T')
turnstile_data = turnstile_data[turnstile_data.index.to_series().between(min(dates), max(dates))] .drop(columns=["entry_diffs", "exit_diffs"])
turnstile_data_cleaned = turnstile.consolidate_turnstile_data(turnstile_data)

## Last Month
last_month_start = min(dates) - timedelta(weeks=4)
last_month_end = max(dates) - timedelta(weeks=4)
turnstile_data_raw = turnstile._process_raw_data(turnstile.download_turnstile_data(start_date=last_month_start, end_date=last_month_end), group_by=['STATION','LINENAME','UNIT'])
turnstile_data_raw_imputed = turnstile.pre_interpolation_fix(turnstile_data_raw)
turnstile_data_raw_imputed = turnstile_data_raw_imputed.set_index('datetime')
last_month_turnstile_data = turnstile._interpolate(turnstile_data_raw_imputed, group_by=['STATION','LINENAME','UNIT'],  frequency='1T')
last_month_turnstile_data = last_month_turnstile_data[last_month_turnstile_data.index.to_series().between(last_month_start, last_month_end)] .drop(columns=["entry_diffs", "exit_diffs"])
last_month_turnstile_data = last_month_turnstile_data.reset_index()
last_month_turnstile_data['datetime'] = last_month_turnstile_data.datetime + timedelta(weeks=4)
last_month_turnstile_data = last_month_turnstile_data.set_index('datetime')
last_month_turnstile_data_cleaned = turnstile.consolidate_turnstile_data(last_month_turnstile_data)

## Last Year
last_year_start = min(dates) - timedelta(weeks=52)
last_year_end = max(dates) - timedelta(weeks=52)
turnstile_data_raw = turnstile._process_raw_data(turnstile.download_turnstile_data(start_date=last_year_start, end_date=last_year_end), group_by=['STATION','LINENAME','UNIT'])
turnstile_data_raw_imputed = turnstile.pre_interpolation_fix(turnstile_data_raw)
turnstile_data_raw_imputed = turnstile_data_raw_imputed.set_index('datetime')
last_year_turnstile_data = turnstile._interpolate(turnstile_data_raw_imputed, group_by=['STATION','LINENAME','UNIT'],  frequency='1T')
last_year_turnstile_data = last_year_turnstile_data[last_year_turnstile_data.index.to_series().between(last_year_start, last_year_end)] .drop(columns=["entry_diffs", "exit_diffs"])
last_year_turnstile_data_cleaned = turnstile.consolidate_turnstile_data(last_year_turnstile_data)


2020-10-26 05:14:25,114 - root - INFO - Downloading turnstile data
2020-10-26 05:14:27,502 - root - INFO - Cleaning turnstile data
2020-10-26 05:14:31,216 - root - INFO - Start interpolating turnstile data
2020-10-26 05:15:04,086 - root - INFO - Finish interpolating
2020-10-26 05:15:07,891 - root - INFO - Finish concatenating the result
2020-10-26 05:15:10,658 - root - INFO - Downloading turnstile data
2020-10-26 05:15:12,566 - root - INFO - Cleaning turnstile data
2020-10-26 05:15:16,345 - root - INFO - Start interpolating turnstile data
2020-10-26 05:15:47,420 - root - INFO - Finish interpolating
2020-10-26 05:15:51,587 - root - INFO - Finish concatenating the result
2020-10-26 05:15:55,144 - root - INFO - Downloading turnstile data
2020-10-26 05:15:57,001 - root - INFO - Cleaning turnstile data
2020-10-26 05:16:02,074 - root - INFO - Start interpolating turnstile data
2020-10-26 05:16:25,982 - root - INFO - Finish interpolating
2020-10-26 05:16:28,584 - root - INFO - Finish concaten

In [8]:
crosswalk = crosswalk[~(crosswalk.turnstile_station_name == '14TH STREET')]

schedule = heuristics.get_schedule_with_weights(turnstile_data_cleaned,clean_df,stop_order_merged)
schedule_last_month = heuristics.get_schedule_with_weights(last_month_turnstile_data_cleaned,clean_df,stop_order_merged)
schedule_last_year = heuristics.get_schedule_with_weights(last_year_turnstile_data_cleaned,clean_static_schedule,stop_order_merged)

df_merge = heuristics.merge_turnstile_schedule(turnstile_data_cleaned,crosswalk, schedule)
last_month_df_merge = heuristics.merge_turnstile_schedule(last_month_turnstile_data_cleaned,crosswalk, schedule_last_month)
last_year_df_merge = heuristics.merge_turnstile_schedule(last_year_turnstile_data_cleaned,crosswalk, schedule_last_year)


In [9]:
df_merge.head()

,STATION,LINENAME,datetime,estimated_entries,estimated_exits,total_entries,total_exits,modified_linename,hour_x,gtfs_station_name,...,uid,legitimate_trunc,max_trip_length,trip_length,pct_max,time,direction_id,hour_y,entry_weight,exit_weight
0,1 AV,L,2020-10-16 00:00:00,1.701719,1.714170,1.701719,1.714170,L,0,1 Av,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,1 AV,L,2020-10-16 00:01:00,1.687302,1.702138,3.389022,3.416309,L,0,1 Av,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,1 AV,L,2020-10-16 00:02:00,1.672885,1.690107,5.061907,5.106415,L,0,1 Av,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,1 AV,L,2020-10-16 00:03:00,1.658468,1.678075,6.720374,6.784490,L,0,1 Av,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,1 AV,L,2020-10-16 00:04:00,1.644051,1.666043,8.364425,8.450534,L,0,1 Av,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [13]:
## Crowd estimation
crowd_by_station_line = heuristics.get_crowd_by_station_line(df_merge,entry_exit_ratio=1.25)
last_month_crowd_by_station_line = heuristics.get_crowd_by_station_line(last_month_df_merge,entry_exit_ratio=1.25)
last_year_crowd_by_station_line = heuristics.get_crowd_by_station_line(last_year_df_merge,entry_exit_ratio=1.25)


2020-10-26 05:19:03,294 - root - INFO - Starting
2020-10-26 05:19:06,507 - root - INFO - Getting people waiting at stations
2020-10-26 05:19:45,358 - root - INFO - Getting crowd per train
2020-10-26 05:20:42,618 - root - INFO - Finishing
2020-10-26 05:20:42,620 - root - INFO - Starting
2020-10-26 05:20:44,788 - root - INFO - Getting people waiting at stations
2020-10-26 05:21:23,780 - root - INFO - Getting crowd per train
2020-10-26 05:22:38,438 - root - INFO - Finishing
2020-10-26 05:22:38,440 - root - INFO - Starting
2020-10-26 05:22:42,892 - root - INFO - Getting people waiting at stations
2020-10-26 05:23:30,142 - root - INFO - Getting crowd per train
2020-10-26 05:24:52,171 - root - INFO - Finishing


In [14]:
## Aggregating estimates by hour and weekday/weekend
station_to_station['from'] = [re.sub(r'N$|S$','',x) for x in station_to_station['from']]
station_to_station['to'] = [re.sub(r'N$|S$','',x) for x in station_to_station['to']]
clean_stop_routes = station_to_station[['from','line']].rename(columns={'from':'stop_id'})
clean_stop_routes = clean_stop_routes.append(station_to_station[['to','line']].rename(columns={'to':'stop_id'}))
clean_stop_routes = clean_stop_routes.drop_duplicates()
clean_stop_routes = clean_stop_routes[~clean_stop_routes.line.isin(['H','SI'])]
clean_stop_routes['route_stop'] = clean_stop_routes['line'] + '_' + clean_stop_routes['stop_id']

current_avg, current_avg_split = heuristics.get_hourly_averages(crowd_by_station_line, clean_stop_routes)
last_month_avg, last_month_avg_split = heuristics.get_hourly_averages(last_month_crowd_by_station_line, clean_stop_routes)
last_year_avg, last_year_avg_split = heuristics.get_hourly_averages(last_year_crowd_by_station_line, clean_stop_routes)

current_avg.rename(columns={'crowd':'current_crowd'}, inplace=True)
last_month_avg.rename(columns={'crowd':'last_month_crowd'}, inplace=True)
last_year_avg.rename(columns={'crowd':'last_year_crowd'}, inplace=True)

hourly_average_estimates = current_avg.merge(last_month_avg,on=['STATION','route_id','hour'])
hourly_average_estimates = hourly_average_estimates.merge(last_year_avg,on=['STATION','route_id','hour'])

current_avg_split.rename(columns={'crowd':'current_crowd'}, inplace=True)
last_month_avg_split.rename(columns={'crowd':'last_month_crowd'}, inplace=True)
last_year_avg_split.rename(columns={'crowd':'last_year_crowd'}, inplace=True)

hourly_average_estimates_split = current_avg_split.merge(last_month_avg_split,on=['STATION','route_id','hour','direction_id','weekday'])
hourly_average_estimates_split = hourly_average_estimates_split.merge(last_year_avg_split,on=['STATION','route_id','hour','direction_id','weekday'])


In [16]:
hourly_average_estimates_split.to_csv('../public/crowding_by_weekday_direction.csv',index=False)


timestr = min(dates).strftime("%Y%m%d")
timesen = max(dates).strftime("%Y%m%d")
test_time = open('../public/timestamp.txt', 'w')
test_time.write(timestr+'-'+timesen)
test_time.close()